# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install tensorflow

  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.67.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached ml_dtypes-0.4.1-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using 

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [5]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode='nearest'
)
training_set = train_datagen.flow_from_directory(
    'dataset/',
    target_size = (128, 128),
    batch_size = 25,
    class_mode = 'categorical')

Found 3792 images belonging to 4 classes.


### Preprocessing the Test set

In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('testset/',
                                            target_size = (128, 128),
                                            batch_size = 25,
                                            class_mode = 'categorical')

Found 60 images belonging to 4 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [27]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [29]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))

E:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [31]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [33]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [35]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [37]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [39]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [41]:
cnn.add(tf.keras.layers.Dense(units=4, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [43]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [49]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 15)

Epoch 1/15
152/152 ━━━━━━━━━━━━━━━━━━━━ 31s 204ms/step - accuracy: 0.9836 - loss: 0.0483 - val_accuracy: 0.9667 - val_loss: 0.0495
Epoch 2/15
152/152 ━━━━━━━━━━━━━━━━━━━━ 30s 195ms/step - accuracy: 0.9812 - loss: 0.0488 - val_accuracy: 0.9500 - val_loss: 0.1015
Epoch 3/15
152/152 ━━━━━━━━━━━━━━━━━━━━ 33s 215ms/step - accuracy: 0.9886 - loss: 0.0327 - val_accuracy: 0.9667 - val_loss: 0.0556
Epoch 4/15
152/152 ━━━━━━━━━━━━━━━━━━━━ 30s 196ms/step - accuracy: 0.9715 - loss: 0.0816 - val_accuracy: 0.9500 - val_loss: 0.1050
Epoch 5/15
152/152 ━━━━━━━━━━━━━━━━━━━━ 26s 169ms/step - accuracy: 0.9731 - loss: 0.0691 - val_accuracy: 0.9667 - val_loss: 0.0448
Epoch 6/15
152/152 ━━━━━━━━━━━━━━━━━━━━ 26s 173ms/step - accuracy: 0.9918 - loss: 0.0275 - val_accuracy: 1.0000 - val_loss: 0.0275
Epoch 7/15
152/152 ━━━━━━━━━━━━━━━━━━━━ 28s 186ms/step - accuracy: 0.9905 - loss: 0.0244 - val_accuracy: 0.9667 - val_loss: 0.0440
Epoch 8/15
152/152 ━━━━━━━━━━━━━━━━━━━━ 29s 191ms/step - accuracy: 0.9837 - loss: 0

## Part 4 - Making a single prediction

In [99]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('./milho_teste/ferrugem2.webp', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


{'doenca1': 0, 'doenca2': 1, 'doenca3': 2, 'saudavel': 3}

In [101]:
print(result)

[[1. 0. 0. 0.]]
